In [1]:
import pandas as pd

считываем наш файл,создаем 2 столбца и преобразуем datetime to the datetime64[ns] Dtype и создаем новые столбцы с этими данными

In [169]:
views = pd.read_csv("../../datasets/feed-views.log",sep="\t",names=["datetime","user"])

In [170]:
views.datetime = pd.to_datetime(views.datetime)

In [171]:
views['year'] = views['datetime'].dt.year
views['month'] = views['datetime'].dt.month
views['day'] = views['datetime'].dt.day
views['hour'] = views['datetime'].dt.hour
views['minute'] = views['datetime'].dt.minute
views['second'] = views['datetime'].dt.second

In [172]:
views.head()

,datetime,user,year,month,day,hour,minute,second
0,2020-04-17 12:01:08.463179,artem,2020,4,17,12,1,8
1,2020-04-17 12:01:23.743946,artem,2020,4,17,12,1,23
2,2020-04-17 12:27:30.646665,artem,2020,4,17,12,27,30
3,2020-04-17 12:35:44.884757,artem,2020,4,17,12,35,44
4,2020-04-17 12:35:52.735016,artem,2020,4,17,12,35,52


создаем новый столбец daytime,назначаем столбец user как index

In [173]:
bins = [-1, 3, 6, 10, 16, 19, 23]
labels = ['night', 'early morning', 'morning', 'afternoon', 'early evening', 'evening']
views['daytime'] = pd.cut(views['hour'], bins=bins, labels=labels)
views = views.set_index('user')

In [174]:
views.tail()

,datetime,year,month,day,hour,minute,second,daytime
user,,,,,,,,
valentina,2020-05-21 18:45:20.441142,2020,5,21,18,45,20,early evening
maxim,2020-05-21 23:03:06.457819,2020,5,21,23,3,6,evening
pavel,2020-05-21 23:23:49.995349,2020,5,21,23,23,49,evening
artem,2020-05-21 23:49:22.386789,2020,5,21,23,49,22,evening
artem,2020-05-22 10:36:14.662600,2020,5,22,10,36,14,morning


считаем количество элементов в нашем df и считаем колво элементов в каждой категории времени суток

In [175]:
print(views.count())
print(views.daytime.value_counts().rename('daytime'))

datetime    1076
year        1076
month       1076
day         1076
hour        1076
minute      1076
second      1076
daytime     1076
dtype: int64
daytime
evening          509
afternoon        252
early evening    145
night            129
morning           36
early morning      5
Name: daytime, dtype: int64


сортировка по времени в порядке возрастания

In [176]:
sorted_views = views.sort_values(['hour', 'minute', 'second'])
sorted_views.head()

,datetime,year,month,day,hour,minute,second,daytime
user,,,,,,,,
valentina,2020-05-15 00:00:13.222265,2020,5,15,0,0,13,night
valentina,2020-05-15 00:01:05.153738,2020,5,15,0,1,5,night
pavel,2020-05-12 00:01:27.764025,2020,5,12,0,1,27,night
pavel,2020-05-12 00:01:38.444917,2020,5,12,0,1,38,night
pavel,2020-05-12 00:01:55.395042,2020,5,12,0,1,55,night


рассчитываем минимум,где время суток - утро и максимум,где время суток - ночь;приводим пример посещения страницы и считаем моду на час и время суток

In [177]:
print(views[views.daytime == 'night'].hour.max())
print(views[views.daytime == 'morning'].hour.min())

3
8


In [178]:
morning = views[views.daytime == 'morning']
min_morning = morning.hour.min()
print(morning[morning.hour == min_morning].index[0])

alexander


In [179]:
print(views.hour.mode()[0])
print(views.daytime.mode()[0])

22
evening


выводим 3 самых ранних утренних часа,а также 3 самых поздних часа вместе с именем пользователя

In [180]:
print(views.nsmallest(3, ['hour', 'minute', 'second'])[['hour', 'minute', 'second']])
print(views.nlargest(3, ['hour', 'minute', 'second'])[['hour', 'minute', 'second']])

           hour  minute  second
user                           
valentina     0       0      13
valentina     0       1       5
pavel         0       1      27
           hour  minute  second
user                           
alexander    23      59      38
valentina    23      58      56
pavel        23      54      54


считаем какой интервал посещенения является самым популярным,считаем межквартельный диапазон

In [181]:
q1 = views.hour.describe()['25%']
q3 = views.hour.describe()['75%']
iqr = q3 - q1
print(iqr)

9.0


тест из чек листа

In [182]:
views.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1076 entries, artem to artem
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  1076 non-null   datetime64[ns]
 1   year      1076 non-null   int32         
 2   month     1076 non-null   int32         
 3   day       1076 non-null   int32         
 4   hour      1076 non-null   int32         
 5   minute    1076 non-null   int32         
 6   second    1076 non-null   int32         
 7   daytime   1076 non-null   category      
dtypes: category(1), datetime64[ns](1), int32(6)
memory usage: 75.6+ KB
